# this script takes the output of stage 1 and prepare a dataset that pytorch likes
# the output of stage 1 is the segmented PPG (8s, no overlap) few minutes before and after the SaO2 sampling time
# 

In [1]:
import pandas as pd
from datetime import datetime
import os
import pickle
from datetime import timedelta
from datetime import datetime
from datetime import date
from datetime import time
from dateutil.relativedelta import relativedelta
import argparse

import os
import math

import pandas as pd
import numpy as np
import os

import json

import sys
from scipy.io import loadmat

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
matplotlib.rc( 'savefig', facecolor = 'white' )
from matplotlib import pyplot
import matplotlib.ticker as plticker

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets, models

# %pip install torch-summary
from torchsummary import summary
torch.manual_seed(0)

import seaborn as sns
from matplotlib.cm import ScalarMappable


import sys
sys.path.append('../') # add this line so Data and data are visible in this file
sys.path.append('../../') # add this line so Data and data are visible in this file
sys.path.append('../PhysioMC/') # add this line so Data and data are visible in this file

# from PatchWand import *
from filters import *
from setting import *
# from preprocessing import *
from ECG_module import *
from dataIO import *
from evaluate import *
from stage1_PPG_analysis import *
from plotting_tools import *


from DR_extension.training_util import *
from DR_extension.dataset_util import *
from DR_extension.evaluation_util import *
from DR_extension.models import *
from DR_extension.models_CNNlight import *

from importlib import reload
%load_ext autoreload
%autoreload 2

In [2]:
print(torch.version.cuda)
print(torch.cuda.is_available())

11.3
True


In [3]:
parser = argparse.ArgumentParser(description='feature_learning')
parser.add_argument('--input_folder', metavar='input_folder', help='input_folder',
                    default='../')
parser.add_argument('--output_folder', metavar='output_folder', help='output_folder',
                    default='../')
parser.add_argument('--training_params_file', metavar='training_params_file', help='training_params_file',
                    default='training_params_list.json')


# checklist 3: comment first line, uncomment second line
args = parser.parse_args(['--input_folder', '../../data/stage1/waveform/', 
                          '--output_folder', '../../data/stage3_DL_prepare_new/',
#                           '--training_params_file', 'training_params_baseline.json',
                          '--training_params_file', 'training_params_dummy.json',
                         ])
# args = parser.parse_args()
print(args)

Namespace(input_folder='../../data/stage1/waveform/', output_folder='../../data/stage3_DL_prepare_new/', training_params_file='training_params_dummy.json')


In [4]:
# selected_id = 10

In [5]:
%pwd

'/labs/kamaleswaranlab/Sepsis3_062321/mchan_analysis/disparities_O2/repo/stage3_DL_RepLearn'

In [6]:
# %pip install statannot


In [7]:
# inputdir = '../../data/stage1/waveform/'
inputdir = args.input_folder
outputdir = args.output_folder
training_params_file = args.training_params_file

# outputdir = '../../data/stage3_DL_RepLearn/'
if not os.path.exists(outputdir):
    os.makedirs(outputdir)


In [8]:
def get_model_out_names(training_params):
    model_out_names = []

#     for output_name in training_params['output_names']:
    for output_name in training_params['output_names']+['domain']:
        for input_name in training_params['input_names']:
            model_out_names.append(output_name+'-{}'.format(input_name))
    return model_out_names

In [9]:
# inputdir

In [10]:
# aaa = data_loader(filename.split('.')[0], inputdir)

In [11]:
# aaa.shape

In [12]:
# 2000/8

In [13]:
# plt.plot(aaa[:,5])
# plt.show()

In [18]:
df_demographic_processed = pd.read_feather(inputdir+'df_demographic_processed.feather', columns=None, use_threads=True)
df_demographic_processed.reset_index(drop=True).to_feather(outputdir+'df_demographic_processed.feather')


In [19]:
# df_demographic_processed = pd.read_csv(inputdir+'df_demographic_processed.csv.gz')  
# df_demographic_processed.to_csv(outputdir+'df_demographic_processed.csv.gz', index=False, compression='gzip')


In [20]:
df_demographic_processed.shape[0]

76450

In [16]:
# mask = df_demographic_processed['debug_message']=='[perfecto]'

In [17]:
# for i, (index, row) in enumerate(df_demographic_processed.iterrows()):
#     pass

In [18]:
# print(os.path.getsize(inputdir+'PPGmatrix_A140-0515603029_0000_0.npz'))


In [19]:
# PPGmatrix = data_loader(row['matrix_name'], inputdir)


In [20]:
# len(os.listdir(inputdir))

In [21]:
list_label = ['spo2', 'sao2', 'Race String', 'PAT_ID']


In [22]:
def parse_samples(inputdir, list_label, df_demographic_processed):


    # df_demographic_processed = pd.read_csv(inputdir+'df_demographic_processed.csv.gz')  

    data = []
    label = []

    for i, (index, row) in enumerate(df_demographic_processed.iterrows()):
        
        if i%1000==0:
            print('{}th sample'.format(i))
        if row['debug_message']!='[perfecto]':
            continue
        # get a waveform (8s, Fs=100Hz)
        PPGmatrix = data_loader(row['matrix_name'], inputdir)
        # select one
        PPGsig = PPGmatrix[:,np.random.randint(PPGmatrix.shape[1])]

        # stack to waveform matrix
        data.append(PPGsig)

        # get the label array
        # stack to label matrix
        label.append(row[list_label].values)

    data = np.stack(data)
    label = np.stack(label)
    
    conversion_rate = data.shape[0]/df_demographic_processed.shape[0]
    print('conversion rate: {:.1f}%'.format(conversion_rate*100) )
    
    return data, label, conversion_rate


In [ ]:
# can take 2 hours for 10k samples

In [23]:
data, label, conversion_rate = parse_samples(inputdir, list_label, df_demographic_processed)


0th sample
1000th sample
2000th sample
3000th sample
4000th sample
5000th sample
6000th sample
7000th sample
8000th sample
9000th sample
10000th sample
11000th sample
12000th sample
13000th sample
14000th sample
15000th sample
16000th sample
17000th sample
18000th sample
19000th sample
20000th sample
21000th sample
22000th sample
23000th sample
24000th sample
25000th sample
26000th sample
27000th sample
28000th sample
29000th sample
30000th sample
31000th sample
32000th sample
33000th sample
34000th sample
35000th sample
36000th sample
37000th sample
38000th sample
39000th sample
40000th sample
41000th sample
42000th sample
43000th sample
44000th sample
45000th sample
46000th sample
47000th sample
48000th sample
49000th sample
50000th sample
51000th sample
52000th sample
53000th sample
54000th sample
55000th sample
56000th sample
57000th sample
58000th sample
59000th sample
60000th sample
61000th sample
62000th sample
63000th sample
64000th sample
65000th sample
66000th sample
67000th 

In [24]:
def get_indices_set(ids_arr, data_arr):
    indices_set = []
    for id_sub in ids_arr:
        indices_set.append(np.where(data_arr==id_sub)[0])
    return np.concatenate(indices_set)

def split_matrix(data, label, indices_set, set_name='null'):

    data = data[indices_set,:]
    label = label[indices_set,:]
    
    return data, label

def save_matrix(data, label, outputdir, set_name='null'):
    outputdir_set = outputdir+set_name+'/'

    if not os.path.exists(outputdir_set):
        os.makedirs(outputdir_set)

    data_saver(data, 'data', outputdir_set)
    data_saver(label, 'label', outputdir_set)

In [25]:
ids_all = np.unique(label[:, -1])
N_sub = ids_all.shape[0]

np.random.seed(i_seed)
np.random.shuffle(ids_all)

ids_TRAIN = ids_all[:int(N_sub*.7)]
ids_TEST = ids_all[int(N_sub*.7):]

# interpolated datapoints
indices_TRAIN = get_indices_set(ids_TRAIN, label[:, -1])
indices_TEST = get_indices_set(ids_TEST, label[:, -1])

print('all ids_TRAIN', ids_TRAIN)
print('all ids_TEST', ids_TEST)
# print('will save to', outputdir)

split_string = np.empty(data.shape[0], dtype=object)
split_string[indices_TEST] = 'TEST'

# print(data_TRAIN.shape, label_TRAIN.shape, data_TEST.shape, label_TEST.shape, )

all ids_TRAIN [73480623.0 19312553.0 98103432.0 97089353.0 19436462.0 98787743.0
 54100639.0 20640827.0 19842177.0 20904667.0 97859403.0 36156984.0
 18519592.0 92188717.0 17965902.0 92490977.0 19447301.0 97653051.0
 21356796.0 98067166.0 36329109.0 98380222.0 96308703.0 98127026.0
 92860511.0 98453868.0 38378004.0 19117138.0 62364416.0 97793074.0
 65068059.0 17673991.0 91467072.0 98803800.0 64157912.0 97138527.0
 17404872.0 97708915.0 99079771.0 87350242.0 65614444.0 98419798.0
 97727051.0 52106922.0 98639605.0 97663818.0 99548188.0 98567550.0
 19075716.0 99005226.0 21276391.0 19161013.0 19316064.0 41218734.0
 34757419.0 24098042.0 95082807.0 54787439.0 94731333.0 17424575.0
 46154427.0 95661380.0 38061020.0 99078675.0 99070642.0 97780479.0
 40561264.0 18366736.0 98772987.0 98228128.0 90981321.0 98468663.0
 97966859.0 98390195.0 99822591.0 17227955.0 46724501.0 34250416.0
 17522047.0 18148343.0 17675200.0 95572378.0 41003140.0 60784059.0
 99519764.0 96564957.0 17653723.0 98348081.0 184

In [26]:
from sklearn.model_selection import KFold

CV_n = 5
rep_n = 1

kfold = CV_n

for i_rep in range(rep_n):

    print('\n')
    kf = KFold(n_splits=kfold, shuffle=True, random_state=i_seed+i_rep)
    kf.get_n_splits(ids_TRAIN)
    print(kf)  

    for i_CV, (train_idx, val_idx) in enumerate(kf.split(ids_TRAIN)):
        print('----------------Splitting for rep {}, CV {}----------------'.format(i_rep, i_CV))
        print("Sub ID | TRAIN:", np.sort(ids_TRAIN[train_idx]).shape, "VAL:", np.sort(ids_TRAIN[val_idx]).shape)
        print('index CV', CV_n*i_rep+i_CV)

        ids_train = ids_TRAIN[train_idx]
        ids_val = ids_TRAIN[val_idx]

        indices_CV = get_indices_set(ids_val, label[:, -1])
        split_string[indices_CV] = 'TRAIN-CV{}'.format(i_CV)
        
        
label = np.c_[label, split_string]
list_label.append('split_name')



KFold(n_splits=5, random_state=0, shuffle=True)
----------------Splitting for rep 0, CV 0----------------
Sub ID | TRAIN: (774,) VAL: (194,)
index CV 0
----------------Splitting for rep 0, CV 1----------------
Sub ID | TRAIN: (774,) VAL: (194,)
index CV 1
----------------Splitting for rep 0, CV 2----------------
Sub ID | TRAIN: (774,) VAL: (194,)
index CV 2
----------------Splitting for rep 0, CV 3----------------
Sub ID | TRAIN: (775,) VAL: (193,)
index CV 3
----------------Splitting for rep 0, CV 4----------------
Sub ID | TRAIN: (775,) VAL: (193,)
index CV 4


In [27]:
data_saver(data, 'data', outputdir)
data_saver(label, 'label', outputdir)

In [28]:
dataset_dict = {}

dataset_dict['list_label'] = list_label
dataset_dict['FS_RESAMPLE'] = FS_RESAMPLE
dataset_dict['window_duration'] = data.shape[1]/FS_RESAMPLE

dataset_dict

{'list_label': ['spo2', 'sao2', 'Race String', 'PAT_ID', 'split_name'],
 'FS_RESAMPLE': 100,
 'window_duration': 8.0}

In [29]:
outputdir

'../../data/stage3_DL_prepare/'

In [30]:
data_saver(dataset_dict, 'dataset_dict', outputdir)


In [31]:
print('done')

done


In [32]:
sys.exit()

SystemExit: 

/opt/rh/rh-python38/root/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3348: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
np.unique(label[:,-1]).shape

In [ ]:
plt.scatter(label[:,0], label[:,1])

In [ ]:
df_demographic_processed = pd.read_csv(inputdir+'df_demographic_processed.csv.gz')  
df_demographic_processed

In [ ]:
np.unique(label[:,-1]).shape

In [ ]:
data_train = data[indices_train, :]
label_train = label[indices_train, :]

data_val = data[indices_val, :]
label_val = label[indices_val, :]

data_val = data[indices_val, :]
label_val = label[indices_val, :]


In [ ]:
data_train, label_train = get_samples(inputdir, 'train/', training_params)
data_val, label_val = get_samples(inputdir, 'val/', training_params)


In [ ]:
data.shape

In [ ]:
plt.plot(data[np.random.randint(data.shape[0]), :])

In [ ]:
for filename in os.listdir(inputdir):
    pass

In [ ]:
with open(training_params_file) as json_file:
    training_params_list = json.load(json_file)

for training_params in [training_params_list[0]]:
    # include device in training_params
    device = torch.device('cuda:{}'.format(int(training_params['cuda_i'])) if torch.cuda.is_available() else 'cpu')
    training_params['device'] = device
    
    if 'training_mode' in training_params:
        training_mode = training_params['training_mode']
    else:
        training_params = 'subject_ind'

    training_params['CV_config'] = {
        'subject_id': 113,
#         'task_id': 5,
    }
    # stage3_dict = data_loader('stage3_dict', inputdir).item()
    # training_params['list_signal'] = stage3_dict['list_signal']
    # training_params['list_feature'] = stage3_dict['list_feature']
    # training_params['list_output'] = stage3_dict['list_output']
    # training_params['list_meta'] = stage3_dict['list_meta']
    training_params['FS_RESAMPLE_DL'] = 100
    # training_params['subject_ids'] = stage3_dict['subject_ids']
    # training_params['task_ids'] = stage3_dict['task_ids']
    # training_params['sequence'] = stage3_dict['sequence']
    
    dataloaders, dataset_sizes = get_loaders(inputdir, training_params)
    print('data dimensions are:', dataloaders['train'].dataset.data.shape)

    data_dimensions = dataloaders['train'].dataset.__getitem__(0)[0].size()
    training_params['data_dimensions'] = list(data_dimensions)
    del dataloaders

    sweep_name = training_params['sweep_name'] 
    
    if training_params['model_name'] == 'FeatureExtractor_CNN':
        training_params['featrue_extractor'] = FeatureExtractor_CNN
    elif training_params['model_name'] == 'ResNet1D':
        training_params['featrue_extractor'] = ResNet1D
    elif training_params['model_name'] == 'FeatureExtractor_CNN2':
        training_params['featrue_extractor'] = FeatureExtractor_CNN2

    training_params['ordered_subject_ids'] = np.asarray(training_params['ordered_subject_ids'])
#       "ordered_subject_ids": [101, 103, 104, 106, 107, 110, 111, 115, 116, 117, 118, 119, 120, 121, 113],

    
    training_params['inputdir'] = inputdir
    training_params['outputdir'] = outputdir
    
    training_params = get_regressor_names(training_params)

    if 'LSTM' in training_params['model_name']:
        training_params = change_output_dim(training_params)

        

    model_out_names = get_model_out_names(training_params)
    training_params['model_out_names'] = model_out_names
    
    
    # integer encode
    label_encoder = LabelEncoder()
    modality_encoded = label_encoder.fit_transform(training_params['input_names'])
#     modality_encoded # array([0, 1])

    modality_dict = {}
    for i_modality in modality_encoded:
        modality_dict[training_params['input_names'][i_modality]] = i_modality
    training_params['modality_dict'] = modality_dict

    training_params['FS_Extracted'] = training_params['FS_RESAMPLE_DL'] / (training_params['stride']**training_params['n_block'])

    
    
    last_layer_dim = training_params['data_dimensions'][-1]
    for n in range(training_params['n_block']):
        last_layer_dim = round(last_layer_dim/training_params['stride'])

    training_params['last_layer_dim'] = last_layer_dim
    xf = np.linspace(0.0, 1.0/2.0*training_params['FS_Extracted'] , training_params['last_layer_dim']//2)*60    
    mask = (xf>=label_range_dict['HR_DL'][0]) & (xf<=label_range_dict['HR_DL'][1])

    training_params['xf'] = xf
    training_params['xf_masked'] = xf[mask]
    training_params['mask'] = mask

# training_params = training_params_list[0]

# test model

In [ ]:
def test_model_lstm(training_params):
    print('test_model_lstm')

    print('using model ', training_params['model_name'])

    # prepare model
    model = resp_multiverse(training_params=training_params)
    model = model.to(device).float()

    # prepare data
    dataloaders, dataset_sizes = get_loaders(inputdir, training_params)

    data = dataloaders['val'].dataset.data[:5,:,:]
    data = torch.from_numpy(data)

    feature = dataloaders['val'].dataset.feature[:5,:]
    feature = torch.from_numpy(feature)

    label = dataloaders['val'].dataset.label[:5,:]
    label = torch.from_numpy(label)

    data = data.to(device=device, dtype=torch.float)
    feature = feature.to(device=device, dtype=torch.float)
    label = label.to(device=device, dtype=torch.float)

    # model inference
    out = model(data, feature)

    # compute loss
    criterion = MultiTaskLoss(training_params)
    losses = criterion(out, label)

    # check losses
    print(losses)
    del model

    
def test_model(training_params):
    print('test_model')

    print('using model ', training_params['model_name'])

    model = resp_multiverse(training_params=training_params)
    summary(model, input_size=[tuple(training_params['data_dimensions']), (model.N_features,1)], device='cpu')
    print(model)
    del model
    
def test_model_dann(training_params):
    print('test_model_dann')
    print('using model ', training_params['model_name'])

    model = resp_DANN(training_params=training_params)
    print(model)

    summary(model, input_size=[tuple(training_params['data_dimensions']), (model.N_features,1)], device='cpu')
    del model



debug_model = False
if debug_model==True:
    if 'LSTM' in training_params['model_name']:
        test_model_lstm(training_params)
    elif 'DANN' not in training_params['model_name']:
        test_model(training_params)
    else:
        test_model_dann(training_params)


In [ ]:
sys.exit()

In [ ]:
df_demographic_processed = pd.read_csv(inputdir+'df_demographic_processed.csv.gz')  
df_demographic_processed['deBedTime'].unique().shape

In [ ]:
for index, row in df_demographic_processed.iterrows():
    continue

In [ ]:
waveform_dir = row['waveform_dir']

In [ ]:
df_bed = pd.read_csv(waveform_dir)  


In [ ]:
df_bed

In [ ]:
plt.plot(df_bed['I'][:5000])å

In [ ]:
# /plt.plot(df_bed['SPO2'].values)

In [ ]:
waveform_dir

In [ ]:
sig_segments = df_bed2PPGmatrix(df_bed)


In [ ]:


data_saver(sig_segments, 'sig_segment_A035-0519626285_0000_7581', '../../data/stage1/waveform/')


In [ ]:
data_saver(sig_segments, 'sig_segment_A035-0519626285_0000_7581', '../../data/stage1/waveform/')
sig_segments = data_loader('sig_segment_A035-0519626285_0000_7581', '../../data/stage1/waveform/')


In [ ]:
plt.plot(sig_segments[:,0])

In [ ]:
plt.plot(sig_segments[:,1])

In [ ]:
sig_segments[:,0].std()

In [ ]:
df = get_filt_df(df_bed, Fs)
plt.plot(df['SPO2'].values[-5000:])

In [ ]:
# df['SPO2'].values.std()

In [ ]:
# signal_matrix = 

In [ ]:
# segment_df

In [ ]:
# 
